# Lab #5
### Cindy Chen


**I am using the same dataset that I've used for my other labs, which is the 2020 Home Mortgage Disclosure Act dataset, which has information on all mortgage applications (including applicant age rang, income, and requested loan amount) in the United States including the application outcome.**

In [187]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt

In [188]:
os.chdir("~/Research Project/2020_lar")
#since the dataset is too big to load into pandas, I only load in the first 8000 rows
mortgage = pd.read_csv("2020_lar.txt", sep = "|", header = 0, nrows = 8000)
mortgage.head(10)

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2020,549300HTRLFU2IIL4380,12940,LA,22037.0,2.203795e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,3.0,NaN,NaN,7086,53.82,73800,81.0,1589,2010,28
1,2020,549300HTRLFU2IIL4380,99999,TX,48467.0,4.846795e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,7182,11.04,59100,120.0,2180,3048,29
2,2020,549300HTRLFU2IIL4380,99999,TX,48337.0,4.833795e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4654,12.01,59100,114.0,1753,2709,32
3,2020,549300HTRLFU2IIL4380,99999,TX,48379.0,4.837995e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6083,9.58,59100,113.0,2076,2995,23
4,2020,549300HTRLFU2IIL4380,26420,TX,48291.0,4.829170e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,5394,17.95,80000,89.0,1798,2627,25
5,2020,549300HTRLFU2IIL4380,99999,TX,48249.0,4.824995e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,3.0,NaN,NaN,7209,79.11,59100,128.0,1589,2393,44
6,2020,549300HTRLFU2IIL4380,99999,TX,48363.0,4.836300e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,2897,16.26,59100,117.0,857,2832,34
7,2020,549300HTRLFU2IIL4380,99999,OK,40001.0,4.000138e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4584,59.79,57500,88.0,1351,1945,34
8,2020,549300HTRLFU2IIL4380,99999,LA,22027.0,2.202795e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,3.0,NaN,NaN,2741,57.57,50400,94.0,599,1334,49
9,2020,549300HTRLFU2IIL4380,26420,TX,48201.0,4.820133e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,3.0,NaN,NaN,11288,92.36,80000,92.0,2662,3192,11


To prepare my data for this lab, I must recode the outcome variable.

In [189]:
conditions = [(mortgage['action_taken'] <= 2), (mortgage['action_taken'] == 3), (mortgage['action_taken'] > 4)]

recoded_conditions = [1, 0, 3]
mortgage['action_taken'] = np.select(conditions, recoded_conditions, default = np.nan)
mortgage.head(10)
#recoded 1 means the mortgage was approved
#recoded 0 means that it was denied
#everything else is considered inadmissable (withdrawn applications, pending apps) so I will remove these

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2020,549300HTRLFU2IIL4380,12940,LA,22037.0,2.203795e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,3.0,NaN,NaN,7086,53.82,73800,81.0,1589,2010,28
1,2020,549300HTRLFU2IIL4380,99999,TX,48467.0,4.846795e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,7182,11.04,59100,120.0,2180,3048,29
2,2020,549300HTRLFU2IIL4380,99999,TX,48337.0,4.833795e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4654,12.01,59100,114.0,1753,2709,32
3,2020,549300HTRLFU2IIL4380,99999,TX,48379.0,4.837995e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6083,9.58,59100,113.0,2076,2995,23
4,2020,549300HTRLFU2IIL4380,26420,TX,48291.0,4.829170e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,5394,17.95,80000,89.0,1798,2627,25
5,2020,549300HTRLFU2IIL4380,99999,TX,48249.0,4.824995e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,3.0,NaN,NaN,7209,79.11,59100,128.0,1589,2393,44
6,2020,549300HTRLFU2IIL4380,99999,TX,48363.0,4.836300e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,2897,16.26,59100,117.0,857,2832,34
7,2020,549300HTRLFU2IIL4380,99999,OK,40001.0,4.000138e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4584,59.79,57500,88.0,1351,1945,34
8,2020,549300HTRLFU2IIL4380,99999,LA,22027.0,2.202795e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,3.0,NaN,NaN,2741,57.57,50400,94.0,599,1334,49
9,2020,549300HTRLFU2IIL4380,26420,TX,48201.0,4.820133e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,3.0,NaN,NaN,11288,92.36,80000,92.0,2662,3192,11


In [190]:
#filter only to the rows with a conclusive mortgage outcome
mortgage = mortgage[(mortgage['action_taken'] == 1) | (mortgage['action_taken'] == 0)]

#confirm that we still have a good number of applications to work with
pd.crosstab(index = mortgage["action_taken"], columns = "count")

col_0,count
action_taken,
0.0,983
1.0,5473


To prepare for this lab, I also want to remove any NAs or other data quality issues in my variables, as well as narrow down my data set to certain columns for manageability.

In [191]:
#remove any rows where there are NAs in my proposed independent variables
sub = mortgage.dropna(subset = ['applicant_age',
                                'income',
                                'property_value',
                                'loan_amount',
                                'combined_loan_to_value_ratio',
                                'loan_term',
                               'action_taken'])

#filter to only these columns of interest
sub = sub.loc[:,['applicant_age',
                'income',
                'property_value',
                'loan_amount',
                'combined_loan_to_value_ratio',
                'loan_term',
                'action_taken']]

#under "applicant_age", there is an age range labelled "8888" that does not make sense.  I will filter out any applicants
#whose age is listed as 8888.

print("Before filtering out 8888: ", sub.shape)
sub = sub[sub['applicant_age'] != '8888']

#view how much data remains after I've filtered my data set for NAs and other data issues
print("After filtering out 8888: ", sub.shape)

Before filtering out 8888:  (5886, 7)
After filtering out 8888:  (5375, 7)


In [192]:
pd.crosstab(index = sub['applicant_age'], columns = "count")

col_0,count
applicant_age,
25-34,1170
35-44,1367
45-54,1133
55-64,893
65-74,503
<25,187
>74,122


In [193]:
#we also need to convert certain data types in our new data set so that we can run an analysis
sub['property_value'] = pd.to_numeric(sub['property_value'], errors = "coerce")
sub['loan_term']  = pd.to_numeric(sub['loan_term'], errors = "coerce")
#convert this to numeric but also divide by 0 so that 80% = 0.8 (instead of 8.0)
sub['combined_loan_to_value_ratio'] = pd.to_numeric(sub['combined_loan_to_value_ratio'], errors = "coerce")/100

#I will also recode applicant_age, which is a categorical range (like "<25" or "26 - 34") into a discrete
#numeric scale from 1 to 7.  I am doing this because I want to see if increases in age lead to higher likelihood of
#mortgage approval.

conditions = [(sub['applicant_age'] == "<25"),
              (sub['applicant_age'] == "25-34"),
             (sub['applicant_age'] == "35-44"),
             (sub['applicant_age'] == "45-54"),
             (sub['applicant_age'] == "55-64"),
             (sub['applicant_age'] == "65-74"),
             (sub['applicant_age'] == ">75")]

choices = [1,2,3,4,5,6,7]

sub['applicant_age'] = np.select(conditions, choices, default = np.nan)

sub.head(10)

,applicant_age,income,property_value,loan_amount,combined_loan_to_value_ratio,loan_term,action_taken
0,2.0,33.0,195000.0,165000,0.8365,360.0,0.0
1,2.0,108.0,285000.0,225000,0.7820,360.0,0.0
2,6.0,68.0,265000.0,235000,0.8804,360.0,1.0
3,2.0,93.0,255000.0,235000,0.9110,360.0,1.0
4,6.0,20.0,245000.0,225000,0.9055,360.0,0.0
5,5.0,11.0,145000.0,145000,1.0356,360.0,0.0
6,4.0,125.0,285000.0,305000,1.0604,360.0,1.0
7,2.0,52.0,55000.0,85000,1.7177,240.0,0.0
8,2.0,45.0,195000.0,165000,0.8497,360.0,0.0
9,4.0,59.0,165000.0,135000,0.8518,360.0,0.0


In [194]:
#finally, I am going to log transform the property value and income variables
sub['property_value'] = np.where(sub['property_value'] != 0, np.log(sub['property_value']), 0)
sub['income'] = np.where(sub['income'] != 0, np.log(sub['income']), 0)

sub.rename(columns = {'property_value':'property_value_log', 'income':'income_log'}, inplace = True)

#let's view the final data frame
sub.head(10)

C:\Users\Cindy C\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Cindy C\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,applicant_age,income_log,property_value_log,loan_amount,combined_loan_to_value_ratio,loan_term,action_taken
0,2.0,3.496508,12.180755,165000,0.8365,360.0,0.0
1,2.0,4.682131,12.560244,225000,0.7820,360.0,0.0
2,6.0,4.219508,12.487485,235000,0.8804,360.0,1.0
3,2.0,4.532599,12.449019,235000,0.9110,360.0,1.0
4,6.0,2.995732,12.409013,225000,0.9055,360.0,0.0
5,5.0,2.397895,11.884489,145000,1.0356,360.0,0.0
6,4.0,4.828314,12.560244,305000,1.0604,360.0,1.0
7,2.0,3.951244,10.915088,85000,1.7177,240.0,0.0
8,2.0,3.806662,12.180755,165000,0.8497,360.0,0.0
9,4.0,4.077537,12.013701,135000,0.8518,360.0,0.0


### **1. Run a multiple linear probability model (have at least 2 Xs in the model). Tell me how you think your independent variables will affect your dependent variable. Interpret your results. Were your expectations correct? Why or why not?**

For my hypothesis: I believe that the variables I have isolated (age group, income (log), property value (log), loan amount, loan-to-value ratio, and loan term length) all affect whether you get your mortgage approved.  This is because the bank has more confidence that someone with accumulated wealth (due to their age group and income) is more likely to pay the loan back and will be willing to approve a longer-term loan since it means that someone has more time to make payments (and it's more profitable for the bank anyway since they'd accrue more interest).  Alternatively, I think property value, loan amount, and loan-to-value might have a strong but negative relationship with loan approval; the larger these values are, the more risky you are as a borrower and therefore the less likely you'll be approved.

From the summary below, I can see that the coefficients for all my variables are statistically significant in relation to whether the mortgage was approved if I use a p-value threshold of 0.05. For instance, the larger the % increase in income or property value, or the longer the loan term, the more likely you'll be approved for a mortgage.  Meanwhile, the lower the loan-to-value ratio (in other words, you made a larger downpayment to offset the loan) and loan amount, the more likely you'll be approved.  These findings align with my initial hypothesis.

However, I am surprised that the coefficient is negative for applicant_age, since I would have thought that graduating into an older age group would correlate with approval.  Perhaps younger applicants are more careful about preparing their first mortgage application and focused on submitting a successful application, while older applicants are riskier with seeing what size mortgage they can get away with.  Finally, although loan amount has a negative coefficient like I had predicted, the coefficient value is extremely close to zero, which suggests that a dollar increase in the loan has very little impact on the approval.

In [195]:
lm1 = smf.ols(
    formula = 'action_taken ~ applicant_age + income_log + property_value_log + loan_amount + combined_loan_to_value_ratio + loan_term',
             data = sub).fit()
print(lm1.summary())

                            OLS Regression Results                            
Dep. Variable:           action_taken   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     45.28
Date:                Mon, 22 Nov 2021   Prob (F-statistic):           2.84e-54
Time:                        22:35:52   Log-Likelihood:                -1581.5
No. Observations:                5106   AIC:                             3177.
Df Residuals:                    5099   BIC:                             3223.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

### **2. Run a multiple (binary) logistic model. (It can be the same as the above LPM or a new model.) If it is a new model, tell me how you think your independent variables will affect your dependent variable. Interpret your results in the logit scale. Were your expectations correct? Why or why not?**

I have run this logit model with the same variables, and they are all statistically significant (though property_value_log is right at the edge of being unable to reject our null hypothesis).  The first thing I noticed is that the coefficients were larger in magnitude compared to my linear probability model (usually a ten-fold increase). This suggests that for a percent increase in income or property value, or a month increase in the loan term, it is even more likely that an applicant will be approved.

We can also see that the applicant_age coefficient remains negative and loan_amount is still very close to 0 so maybe my explanation in the previous section about why the applicant_age coefficient is negative has merit.

In [196]:
logit1 = sm.formula.logit(formula = 'action_taken ~ applicant_age + income_log + property_value_log + loan_amount + combined_loan_to_value_ratio + loan_term',
                        data = sub).fit()

print(logit1.summary())

Optimization terminated successfully.
         Current function value: 0.365568
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:           action_taken   No. Observations:                 5106
Model:                          Logit   Df Residuals:                     5099
Method:                           MLE   Df Model:                            6
Date:                Mon, 22 Nov 2021   Pseudo R-squ.:                 0.06306
Time:                        22:35:52   Log-Likelihood:                -1866.6
converged:                       True   LL-Null:                       -1992.2
Covariance Type:            nonrobust   LLR p-value:                 2.191e-51
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -5.3533      1.863     -2.873      0.004

### **3. Get odds ratios from your logit model in Question 2 and interpret some of them.**

Net of other factors, every 1% increase in income will lead to a 140% increase in odds that an applicant will get approved for a mortgage. In addition, a 1% increase in income will lead to 35% better odds of getting approved for your mortgage. Meanwhile, a month increase in the loan term or a dollar increase in the loan amount doesn't have a material effect on the odds. Moreover, a point increase in the combined loan/value ratio reduces the odds by 75%, and an increase in the applicant_age group will make the odds of mortgage approval 10% less likely to occur.

In [197]:
np.exp(logit1.params)

Intercept                       0.004733
applicant_age                   0.904406
income_log                      2.404644
property_value_log              1.353176
loan_amount                     0.999996
combined_loan_to_value_ratio    0.249929
loan_term                       1.006432
dtype: float64

### **4. Get predicted probabilities from your logit model in Question 2 for some constellations of X values and interpret the results.**

In [198]:
#let me view the descriptive statistics for my subsetted data frame so I can select values to predict the probabilities
sub.describe()

,applicant_age,income_log,property_value_log,loan_amount,combined_loan_to_value_ratio,loan_term,action_taken
count,5253.000000,5374.000000,5227.000000,5.375000e+03,5227.000000,5227.000000,5375.000000
mean,3.549020,4.441478,12.502805,2.237330e+05,0.764378,318.119380,0.870884
std,1.349637,0.655151,0.564412,1.327731e+05,0.227544,81.614799,0.335360
min,1.000000,0.000000,9.615805,5.000000e+03,0.018900,8.000000,0.000000
25%,2.000000,4.025352,12.180755,1.250000e+05,0.655890,360.000000,1.000000
50%,3.000000,4.442651,12.524526,2.050000e+05,0.800000,360.000000,1.000000
75%,5.000000,4.852030,12.911642,2.950000e+05,0.950000,360.000000,1.000000
max,6.000000,7.783224,14.846914,1.285000e+06,4.084970,380.000000,1.000000


In [199]:
def logit2prob (logit):
    odds = np.exp(logit)
    prob = odds / (1 + odds)
    return(prob);


intercept = logit1.params.Intercept
b_applicant_age = logit1.params.applicant_age
b_income_log = logit1.params.income_log
b_property_value_log = logit1.params.property_value_log
b_loan_amount = logit1.params.loan_amount
b_combined_loan_to_value_ratio = logit1.params.combined_loan_to_value_ratio
b_loan_term = logit1.params.loan_term

For an applicant who is under 25, has income in the lowest quartile, is trying to buy a home in the bottom quartile, requests a $60,000 mortgage, wants to assume a combined loan to value ratio of 0.5, and requests a 10-year mortgage, their odds of getting their mortgage approved is 83.2 probability.

In [200]:
logits_youngapplicant = intercept + (1 * b_applicant_age) + (4.02535 * b_income_log) + (12.180755 * b_property_value_log) + (60000 * b_loan_amount) + (0.5 * b_combined_loan_to_value_ratio) + (120 * b_loan_term)

print("young applicant: {:.3f}".format(logit2prob(logits_youngapplicant)))

young applicant: 0.832


For an applicant between the ages of 45 and 54, has average income, is trying to buy a home in the top quartile, requests a $900,000 mortgage, wants to assume a combined loan to value ratio of 0.8, and requests a 30-year mortgage, their odds of getting their mortgage approved is 40.5 probability.  This makes sense these stats are much riskier.

In [201]:
logits_risky_applicant = intercept + (4 * b_applicant_age) + (4.441478 * b_income_log) + (12.911642 * b_property_value_log) + (900000 * b_loan_amount) + (0.8 * b_combined_loan_to_value_ratio) + (360 * b_loan_term)

print("risky applicant: {:.3f}".format(logit2prob(logits_risky_applicant)))

risky applicant: 0.405
